# T5 - The Kalman filter (KF) -- multivariate
Dealing with vectors and matrices is a lot like plain numbers. But some things get more complicated.
We begin by illustrating Bayes' rule in the 2D, i.e. multivariate, case.
$
% START OF MACRO DEF
% DO NOT EDIT IN INDIVIDUAL NOTEBOOKS, BUT IN macros.py
%
\newcommand{\Reals}{\mathbb{R}}
\newcommand{\Expect}[0]{\mathbb{E}}
\newcommand{\NormDist}{\mathcal{N}}
%
\newcommand{\DynMod}[0]{\mathscr{M}}
\newcommand{\ObsMod}[0]{\mathscr{H}}
%
\newcommand{\mat}[1]{{\mathbf{{#1}}}}
%\newcommand{\mat}[1]{{\pmb{\mathsf{#1}}}}
\newcommand{\bvec}[1]{{\mathbf{#1}}}
%
\newcommand{\trsign}{{\mathsf{T}}}
\newcommand{\tr}{^{\trsign}}
\newcommand{\tn}[1]{#1}
\newcommand{\ceq}[0]{\mathrel{≔}}
%
\newcommand{\I}[0]{\mat{I}}
\newcommand{\K}[0]{\mat{K}}
\newcommand{\bP}[0]{\mat{P}}
\newcommand{\bH}[0]{\mat{H}}
\newcommand{\bF}[0]{\mat{F}}
\newcommand{\R}[0]{\mat{R}}
\newcommand{\Q}[0]{\mat{Q}}
\newcommand{\B}[0]{\mat{B}}
\newcommand{\C}[0]{\mat{C}}
\newcommand{\Ri}[0]{\R^{-1}}
\newcommand{\Bi}[0]{\B^{-1}}
\newcommand{\X}[0]{\mat{X}}
\newcommand{\A}[0]{\mat{A}}
\newcommand{\Y}[0]{\mat{Y}}
\newcommand{\E}[0]{\mat{E}}
\newcommand{\U}[0]{\mat{U}}
\newcommand{\V}[0]{\mat{V}}
%
\newcommand{\x}[0]{\bvec{x}}
\newcommand{\y}[0]{\bvec{y}}
\newcommand{\z}[0]{\bvec{z}}
\newcommand{\q}[0]{\bvec{q}}
\newcommand{\br}[0]{\bvec{r}}
\newcommand{\bb}[0]{\bvec{b}}
%
\newcommand{\bx}[0]{\bvec{\bar{x}}}
\newcommand{\by}[0]{\bvec{\bar{y}}}
\newcommand{\barB}[0]{\mat{\bar{B}}}
\newcommand{\barP}[0]{\mat{\bar{P}}}
\newcommand{\barC}[0]{\mat{\bar{C}}}
\newcommand{\barK}[0]{\mat{\bar{K}}}
%
\newcommand{\D}[0]{\mat{D}}
\newcommand{\Dobs}[0]{\mat{D}_{\text{obs}}}
\newcommand{\Dmod}[0]{\mat{D}_{\text{obs}}}
%
\newcommand{\ones}[0]{\bvec{1}}
\newcommand{\AN}[0]{\big( \I_N - \ones \ones\tr / N \big)}
%
% END OF MACRO DEF
$

In [ ]:
import resources.workspace as ws
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.ion();

## Multivariate Bayes' rule

#### Exc 2 (The likelihood):
Suppose the observation, $\y$, is related to the true state, $\x$, via an "observation (forward) model", $\ObsMod$:
\begin{align*}
\y &= \ObsMod(\x) + \br \, , \;\; \qquad (2)
\end{align*}
where the noise follows the law $\br \sim \NormDist(\bvec{0}, \R)$ for some $\R>0$ (meaning $\R$ is symmetric-positive-definite).


Derive the expression for the likelihood, $p(\y|\x)$.

In [ ]:
# ws.show_answer('Likelihood for additive noise')

Unlike [T2](T2%20-%20Gaussian%20distribution.ipynb), which implemented the Gaussian pdf,
we here take it straight from `scipy.stats`.

In [ ]:
from scipy.stats import multivariate_normal
def pdf_GM(points, mu, Sigma):
    diff = points - mu  # broadcasts both
    zero = np.zeros(len(Sigma))
    return multivariate_normal(zero, Sigma).pdf(diff)

The following code shows Bayes' rule in action.
Notice that the implementation (`Bayes_rule`) is the very same as in the 1D case
-- simply the pointwise multiplication of two arrays.

In [ ]:
def Bayes_rule(prior_values, lklhd_values, dx):
    prod = prior_values * lklhd_values         # pointwise multiplication
    posterior_values = prod/(np.sum(prod)*dx)  # normalization
    return posterior_values

bounds = -15, 15
grid1d = np.linspace(*bounds, 201)
dx = grid1d[1]  - grid1d[0]

In [ ]:
grid2d = np.dstack(np.meshgrid(grid1d, grid1d))

@ws.interact(corr_R=(-0.999, 0.999, .01),
             corr_B=(-0.999, 0.999, .01),
             y1=bounds,
             y2=bounds,
             R1=(0.01, 36, 0.2),
             R2=(0.01, 36, 0.2),
             top=[['corr_B', 'corr_R'], 'y1_only'],
             bottom=[['y1', 'R1']],
             vertical=['y2', 'R2'],
             right=[['y2', 'R2']],
             )
def Bayes2(corr_B=.6, corr_R=.6, y1=3, y2=-12, R1=4**2, R2=1, y1_only=False):
    x = grid2d
    # Prior
    mu = np.zeros(2)
    B = 25 * np.array([[1, corr_B],
                       [corr_B, 1]])
    # Likelihood
    cov_R = np.sqrt(R1*R2)*corr_R
    R = np.array([[R1, cov_R],
                  [cov_R, R2]])
    y = np.array([y1, y2])
    Hx = x
    #Hx = x**2/4
    #Hx = x**3/36

    #Hx = x[..., :1] * x[..., 1:2]
    #y1_only = True

    if y1_only:
        y = y[:1]
        R = R[:1, :1]
        Hx = Hx[..., :1]

    # Compute
    lklhd = pdf_GM(y, Hx, R)
    prior = pdf_GM(x, mu, B)
    postr = Bayes_rule(prior, lklhd, dx**2)

    ax, plot = ws.get_jointplotter(grid1d)
    contours = [plot(prior, 'blue'),
                plot(lklhd, 'green'),
                plot(postr, 'red', linewidths=2)]
    ax.legend(contours, ['prior', 'lklhd', 'postr'], loc="upper left")
    plt.show()

#### Exc
- Does the posterior (pdf) generally lie "between" the prior and likelihood?

#### Exc: Observation models
- Implement $\mathcal{H}(\x) = \x_1$.
- Implement $\mathcal{H}(\x) = \frac{1}{M} \sum_{i=1}^M \x_i$.
- Implement $\mathcal{H}(\x) = \x_2 - \x_1$.
- Implement $\mathcal{H}(\x) = (\x_1^2, \x_2^2)$.
- Implement $\mathcal{H}(\x) = \x_1 \x_2$.
- For those of the above models that are linear,
  find the (possibly rectangular) matrix $\bH$ such that $\mathcal{H}(\x) = \bH \x$.

## The KF analysis step

The following exercise derives the analysis step.

#### Exc 4 (The 'precision' form of the KF):
Similarly to [Exc 2.18](T3%20-%20Bayesian%20inference.ipynb#Exc--2.18-'Gaussian-Gaussian-Bayes':),
it may be shown that the prior $p(\x) = \NormDist(\x \mid \bb,\B)$
and likelihood $p(\y|\x) = \NormDist(\y \mid \bH \x,\R)$,
yield the posterior:
\begin{align}
p(\x|\y)
&= \NormDist(\x \mid \hat{\x}, \bP) \tag{4}
\, ,
\end{align}
where the posterior/analysis mean (vector) and covariance (matrix) are given by:
\begin{align}
			\bP &= (\bH\tr \Ri \bH + \Bi)^{-1} \, , \tag{5} \\
			\hat{\x} &= \bP\left[\bH\tr \Ri \y + \Bi \bb\right] \tag{6} \, ,
\end{align}
Prove eqns (4-6).  
*Hint: like the last time, the main part lies in "completing the square" in $\x$.*

In [ ]:
# ws.show_answer('KF precision')

However, the computations can be pretty expensive...

#### Exc 5 (optional): Suppose $\x$ is $M$-dimensional and has a covariance matrix $\B$.
 * (a). What's the size of $\B$?
 * (b). How many "flops" (approximately, i.e. to leading order) are required  
 to compute the "precision form" of the KF update equation, eqn (5) ?
 * (c). How much memory (bytes) is required to hold its covariance matrix $\B$ ?
 * (d). How many megabytes is this if $M$ is a million?

In [ ]:
# ws.show_answer('Cov memory')

This is one of the principal reasons why basic extended KF is infeasible for DA.  
The following derives another, often more practical, form of the KF analysis update.

#### Exc 6 (The "Woodbury" matrix inversion identity):
The following is known as the Sherman-Morrison-Woodbury lemma/identity,
$$\begin{align}
    \bP = \left( \B^{-1} + \V\tr \R^{-1} \U \right)^{-1}
    =
    \B - \B \V\tr \left( \R + \U \B \V\tr \right)^{-1} \U \B \, ,
    \tag{W}
\end{align}$$
which holds for any (suitably shaped matrices)
$\B$, $\R$, $\V,\U$ *such that the above exists*.

Prove the identity. Hint: don't derive it, just prove it!

In [ ]:
# ws.show_answer('Woodbury')

#### Exc 7 (optional):
- Show that $\B$ and $\R$ must be square.
- Show that $\U$ and $\V$ are not necessarily square, but must have the same dimensions.
- Show that $\B$ and $\R$ are not necessarily of equal size.


Exc 7 makes it clear that the Woodbury identity may be used to compute $\bP$ by inverting matrices of the size of $\R$ rather than the size of $\B$.
Of course, if $\R$ is bigger than $\B$, then the identity is useful the other way around.

#### Exc 8 (Corollary 1 -- optional):
Prove that, for any symmetric, positive-definite (SPD) matrices $\R$ and $\B$, and any matrix $\bH$,
$$\begin{align}
 	\left(\bH\tr \R^{-1} \bH + \B^{-1}\right)^{-1}
    &=
    \B - \B \bH\tr \left( \R + \bH \B \bH\tr \right)^{-1} \bH \B \tag{C1}
    \, .
\end{align}$$
Hint: consider the properties of [SPD](https://en.wikipedia.org/wiki/Definiteness_of_a_matrix#Properties) matrices.

In [ ]:
# ws.show_answer('Woodbury C1')

#### Exc 10 (Corollary 2 -- optional):
Prove that, for the same matrices as for Corollary C1,
$$\begin{align}
	\left(\bH\tr \R^{-1} \bH + \B^{-1}\right)^{-1}\bH\tr \R^{-1}
    &= \B \bH\tr \left( \R + \bH \B \bH\tr \right)^{-1}
    \tag{C2}
    \, .
\end{align}$$

In [ ]:
# ws.show_answer('Woodbury C2')

#### Exc 12 (The "gain" form of the KF):
Now, let's go back to the KF, eqns (5) and (6). Since $\B$ and $\R$ are covariance matrices, they are symmetric-positive. In addition, we will assume that they are full-rank, making them SPD and invertible.  

Define the Kalman gain by:
 $$\begin{align}
    \K &= \B \bH\tr \big(\bH \B \bH\tr + \R\big)^{-1} \, . \tag{K1}
\end{align}$$
 * (a) Apply (C1) to eqn (5) to obtain the Kalman gain form of analysis/posterior covariance matrix:
$$\begin{align}
    \bP &= [\I_M - \K \bH]\B \, . \tag{8}
\end{align}$$

* (b) Apply (C2)  to (5) to obtain the identity
$$\begin{align}
    \K &= \bP \bH\tr \R^{-1}  \, . \tag{K2}
\end{align}$$

* (c) Show that $\bP \Bi = [\I_M - \K \bH]$.
* (d) Use (b) and (c) to obtain the Kalman gain form of analysis/posterior covariance
$$\begin{align}
     \hat{\x} &= \bb + \K\left[\y - \bH \bb\right] \, . \tag{9}
\end{align}$$

Together, eqns (8) and (9) define the Kalman gain form of the KF update.
The inversion (eqn 7) involved is of the size of $\R$, while in eqn (5) it is of the size of $\B$.

## The KF forecast step

The forecast step remains essentially unchanged from the [univariate case](T3%20-%20Bayesian%20inference.ipynb).
The only difference is that $\DynMod$ is now a matrix, as well as the use of the transpose ${}^T$ in the covariance equation:
$\begin{align}
\bb_k
&= \DynMod_{k-1} \hat{\x}_{k-1} \, , \tag{1a} \\\
\B_k
&= \DynMod_{k-1} \bP_{k-1} \DynMod_{k-1}^T + \Q_{k-1} \, . \tag{1b}
\end{align}$

<mark><font size="-1">
We have now derived the Kalman filter, also in the multivariate case. We know how to:
</font></mark>
- Propagate our estimate of $\x$ to the next time step using eqns (1a) and (1b).
- Update our estimate of $\x$ by assimilating the latest observation $\y$, using eqns (5) and (6).

## Another time series problem, but this time multivariate
In the previous tutorial we saw that the Kalman filter (KF) works well
for univariate (scalar) time series problem. Let us try it on a multivariate problem.

### The model
- The straight line example (of the tutorial T3) could result from discretizing the model:
$
\frac{d^2 x}{dt^2} = 0 \, .
$
- Here we're going to consider the M-th order model:
$ \frac{d^M x}{dt^M} = 0 \, .$
- This can be rewritten as a 1-st order vector (i.e. coupled system of) ODE:
$\frac{d x_m}{dt} = x_{m+1} \, ,$
where the subscript $1,\ldots,M$ is now instead the *index* of the state vector element,
and $x_{M+1} = 0$.
- To make it more interesting, we'll add two terms to this evolution model:  
    - damping: $\beta x_m$, with $\beta < 0$;
    - noise: $\frac{d q_m}{dt}$.  

Thus,
$$ \frac{d x_m}{dt} = \beta x_m + x_{m+1} + \frac{d q_m}{dt} \, ,$$
where $q_m$ is the noise process, and $\beta = \log(0.9)$.
Discretized by explicit-Euler, with a time step `dt=1`, this yields
$$ x_{k+1, m} = 0.9 x_{k, m} + x_{k, m+1} + q_{k, m}\, ,$$

In summary, $\x_{k+1} = \DynMod \x_k + \q_k$, with $\DynMod$ as below.

In [ ]:
M = 4 # model order (and also ndim)
M_matrix = 0.9*np.eye(M) + np.diag(np.ones(M-1), 1)
print(M_matrix)

### Estimation by the Kalman filter (and smoother) with DAPPER

Note that this is an $M$-dimensional time series.
However, we'll only observe the first (0th) component.

We shall not write the code for the multivariate Kalman filter,
because it already exists in DAPPER in `da_methods.py` and is called `ExtKF()`.

The following code configures an experiment based on the above model. Don't worry about the specifics. We'll get back to how to use DAPPER later.


In [ ]:
import dapper.mods as modelling
from dapper.mods.utils import linear_model_setup, partial_Id_Obs

# Forecast dynamics
Dyn = linear_model_setup(M_matrix, dt0=1)
Dyn['noise'] = 0.0001*(1+np.arange(M))

# Initial conditions
X0 = modelling.GaussRV(M=M, C=0.02*np.arange(M))

# Observe 0th component only
Obs = partial_Id_Obs(M, [0])
Obs['noise'] = 1000

# Time settings
t = modelling.Chronology(dt=1, dto=5, K=250)

# Wrap-up
HMM = modelling.HiddenMarkovModel(Dyn, Obs, t, X0)

This generates (simulates) a synthetic truth (xx) and observations (yy)

In [ ]:
xx, yy = HMM.simulate()
for m, x in enumerate(xx.T):
    plt.plot(x, label="x^%d"%m)
plt.legend();

Now we'll run assimilation methods on the data. Firstly, the KF, available as `ExtKF` in DAPPER:

In [ ]:
import dapper.da_methods as da
ExtKF = da.ExtKF()
ExtKF.assimilate(HMM, xx, yy)

We'll also run the "Kalman smoother" available as `ExtRTS`.
Without going into details, this method is based on the Kalman *filter* but,
being a *smoother*,
it also goes backwards and updates previous estimates with future (relatively speaking) observations.

In [ ]:
ExtRTS = da.ExtRTS()
ExtRTS.assimilate(HMM, xx, yy)

### Estimation by "time series analysis"
The following methods perform time series analysis of the observations, and are mainly derived from signal processing theory.
Considering that derivatives can be approximated by differentials, it is plausible that the above model could also be written as an AR(M) process. Thus these methods should perform quite well.

In [ ]:
# Tools
import scipy as sp
import scipy.signal as sp_sp
normalize = lambda x: x / x.sum()
truncate  = lambda x, n: np.hstack([x[:n], np.zeros(len(x)-n)])

# We only estimate the 0-th component.
signal = yy[:, 0]

# Estimated signals
ESig = {}
ESig['Gaussian'] = sp_sp.convolve(signal, normalize(sp_sp.gaussian(30, 3)), 'same')
ESig['Wiener']   = sp_sp.wiener(signal)
ESig['Butter']   = sp_sp.filtfilt(*sp_sp.butter(10, 0.12), signal, padlen=len(signal)//10)
ESig['Spline']   = sp.interpolate.UnivariateSpline(t.kko, signal, s=1e4)(t.kko)
ESig['Low-pass'] = np.fft.irfft(truncate(np.fft.rfft(signal), len(signal)//14))

### Comparison
The following code plots the results.

In [ ]:
@ws.interact(Visible=ws.SelectMultiple(options=['Truth', *ESig, 'My Method',
                                                'Kalman smoother', 'Kalman filter', 'Kalman filter a']))
def plot_results(Visible):
    plt.figure(figsize=(9, 5))
    plt.plot(t.kko, yy, 'k.', alpha=0.4, label="Obs")
    if 'Truth'           in Visible: plt.plot(t.kk , xx[:, 0]               , 'k', label="Truth")
    if 'Kalman smoother' in Visible: plt.plot(t.kk , ExtRTS.stats.mu.u[:, 0], 'm', label="K. smoother")
    if 'Kalman filter'   in Visible: plt.plot(t.kk , ExtKF .stats.mu.u[:, 0], 'b', label="K. filter")
    if 'Kalman filter a' in Visible: plt.plot(t.kko, ExtKF .stats.mu.a[:, 0], 'b', label="K. filter (a)")
    if 'My Method'       in Visible: plt.plot(t.kk , MyMeth.stats.mu.u[:, 0], 'b', label="My method")
    for method, estimate in ESig.items():
        if method in Visible: plt.plot(t.kko, estimate, label=method)

    plt.ylabel('$x^0$, $y$, and $\hat{x}^0$')
    plt.xlabel('Time index ($k$)')
    plt.legend()
    plt.show()

Visually, it's hard to imagine better performance than from the Kalman smoother.
However, recall the advantage of the Kalman filter (and smoother): *they know the forecast model that generated the truth*.

Since the noise levels Q and R are given to the DA methods (but they don't know the actual outcomes/realizations of the random noises), they also do not need any *tuning*, compared to signal processing filters, or choosing between the myriad of signal processing filters [out there](https://docs.scipy.org/doc/scipy/reference/signal.html#module-scipy.signal).

In [ ]:
def average_error(estimate_at_obs_times):
    return np.mean(np.abs(xx[t.kko, 0] - estimate_at_obs_times))

for method, estimate in {**ESig,
                         'K. smoother': ExtRTS.stats.mu.u[t.kko, 0],
                         'K. filter'  : ExtKF .stats.mu.a[:, 0],
                         # 'My Method'  : MyMeth.stats.mu.a[:, 0], # uncomment after Exc 8
                        }.items():
    print("%20s" % method, "%.4f" % average_error(estimate))

**Exc 14:** Theoretically, in the long run, the Kalman smoother should yield the optimal result. Verify this by increasing the experiment length to `K=10**4`.

**Exc 16:** Re-run the experiment with different parameters, for example the observation noise strength or `dko`.  
[Results will differ even if you changed nothing because the truth noises (and obs) are stochastic.]

**Exc 18:** Right before executing the assimilations (but after simulating the truth and obs), change $R$ by inserting:

    HMM.Obs.noise = GaussRV(C=0.01*np.eye(1))

What happens to the estimates of the Kalman filter and smoother?

**Exc 20 (optional):** Try out different methods from DAPPER by replacing `MyMethod` below with one of the following:
 - Climatology
 - Var3D
 - OptInterp
 - EnKF
 - EnKS
 - PartFilt

You typically also need to set (and possibly tune) some method parameters. Otherwise you will get an error (or possibly the method will perform very badly). You may find (some) documentation for each method in its source code...

In [ ]:
MyMeth = da.MyMethod(param1=val1, ...)
MyMeth.assimilate(HMM, xx, yy)

### Summary
We have derived two forms of the multivariate KF analysis update step: the "precision matrix" form, and the "Kalman gain" form. The latter is especially practical when the number of observations is smaller than the length of the state vector.

As a subset of state estimation we can do time series estimation
[(wherein state-estimation is called state-space approach)](https://www.google.com/search?q="We+now+demonstrate+how+to+put+these+models+into+state+space+form").
Moreover, DA methods produce uncertainty quantification, something which is usually more obscure with time series analysis methods.
Still, the best is yet to come: the ability to handle very large and chaotic systems
(which are more fun than stochastically driven signals such as above).

### Next: [T6 - Spatial statistics ("geostatistics") & Kriging](T6%20-%20Geostatistics%20%26%20Kriging.ipynb)